In [1]:
import agents_env
from agents_env.agent_mimic_train import HumanoidEnvTrain
from utils.SimpleConverter import SimpleConverter
from utils.util_data import *
from copy import deepcopy

In [2]:
from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import train as ppo
from brax.training.agents.ppo import networks as ppo_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap
import jax.random
from jax import lax

In [3]:
#this is just dummy data to initialize the agent
trajectory = SimpleConverter('motions/humanoid3d_punch.txt')
trajectory.load_mocap()
model_path = 'models/final_humanoid.xml'

data_mocap_matrix = jp.asarray(trajectory.data)
data_pos_mocap = jp.asarray(trajectory.data_pos)
data_vel_mocap = jp.asarray(trajectory.data_vel)

data_dict_mocap = trajectory.duration_dict

data_xpos_mocap=jp.asarray(trajectory.data_xpos)
data_xrot_mocap=jp.asarray(trajectory.data_xrot)

In [4]:
#get th kp and kd for the agent
kp,kd = generate_kp_kd_gains()
print(kp)
print(kd)

[1000 1000 1000  100  100  100  400  400  400  300  400  400  400  300
  500  500  500  500  400  400  400  500  500  500  500  400  400  400]
[100 100 100  10  10  10  40  40  40  30  40  40  40  30  50  50  50  50
  40  40  40  50  50  50  50  40  40  40]


In [5]:
import yaml
from box import Box
# Path to your YAML file
yaml_file_path = 'config_params/punch.yaml'
# Load the YAML file
with open(yaml_file_path, 'r') as file:
    args = Box(yaml.safe_load(file))

In [6]:
envs.register_environment('humanoidEnvMimic',HumanoidEnvTrain)
env_name = 'humanoidEnvMimic'
env = envs.get_environment(env_name=env_name,
                           reference_trajectory_qpos=data_pos_mocap,
                           reference_trajectory_qvel = data_vel_mocap,
                            duration_trajectory=trajectory.total_time,
                            dict_duration= data_dict_mocap,
                           model_path=model_path,
                           kp_gains = kp,
                           kd_gains = kd,
                           reference_x_pos=data_xpos_mocap,
                           reference_x_rot=data_xrot_mocap,
                           args=args)
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step_custom)


#select the pd_control
from agents_env.pds_controllers_agents import stable_pd_controller_custom_trajectory, stable_pd_controller_trajectory,stable_pd_controller_action,feedback_pd_controller
#env.set_pd_callback(stable_pd_controller_custom_trajectory)
env.set_pd_callback(feedback_pd_controller)
#env.set_pd_callback(feedback_pd_controller)

In [7]:
args

Box({'num_envs': 256, 'num_eval_envs': 32, 'lr': 0.0003, 'max_it': 1000, 'max_grad_norm': 0.4, 'seed': 0, 'system_config': 'humanoid', 'demo_replay_mode': 'threshold', 'threshold': 0.4, 'normalize_observations': True, 'cycle_len': 0, 'ep_len': 30, 'ep_len_eval': 54, 'use_lr_scheduler': True, 'reward_scaling': 0.02, 'rot_weight': 0.5, 'vel_weight': 0.01, 'ang_weight': 0.01, 'ref': 'motions/humanoid3d_punch.txt'})

In [8]:
print(env.rollout_lenght)
print(env.cycle_len)
print(env.rot_weight)
print(env.vel_weight)
print(env.ang_weight)
print(env.reward_scaling)
#print(env.dict_duration)

65
65
0.5
0.01
0.01
0.02


In [9]:
# initialize the state
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]
# grab a 500 steps
n_steps = 500

In [10]:
idx = ( 64 % env.rollout_lenght) 

phi = ( 66 % env.rollout_lenght) / env.rollout_lenght

print(phi)
print(idx)

0.015384615384615385
64


In [11]:

mask = jp.where(65 <= env.rollout_lenght-1, jp.float32(54), 0)
print(mask)


0.0


In [12]:

for i in range(env.rollout_lenght):
    
    print(i)
    ctrl = -0.1 * jp.ones(env.sys.nu)
    #time
    time = state.pipeline_state.time
    
    #print('time: ',state.pipeline_state.time)    
         
    state = jit_step(state, ctrl)
   
    
    #print("Is done", state.done)
    #print("Is rewards", state.reward)
    
    rollout.append(state.pipeline_state)

0
pose_error: 0.0
1
pose_error: 0.0
2
pose_error: 0.0
3
pose_error: 0.0
4
pose_error: 0.0
5
pose_error: 0.0
6
pose_error: 0.0
7
pose_error: 0.0
8
pose_error: 0.0
9
pose_error: 0.0
10
pose_error: 0.0
11
pose_error: 0.0
12
pose_error: 0.0
13
pose_error: 0.0
14
pose_error: 0.0
15
pose_error: 0.0
16
pose_error: 0.0
17
pose_error: 0.0
18
pose_error: 0.0
19
pose_error: 7.143896141315054e-08
20
pose_error: 0.0
21
pose_error: 0.0
22
pose_error: 0.0
23
pose_error: 0.0
24
pose_error: 0.0
25
pose_error: 0.0
26
pose_error: 1.2902424373351096e-07
27
pose_error: 0.0
28
pose_error: 0.0
29
pose_error: 0.0
30
pose_error: 0.0
31
pose_error: 0.0
32
pose_error: 0.0
33
pose_error: 0.0
34
pose_error: 0.0
35
pose_error: 0.0
36
pose_error: 0.0
37
pose_error: 0.0
38
pose_error: 0.0
39
pose_error: 1.0072823641849027e-07
40
pose_error: 0.0
41
pose_error: 0.0
42
pose_error: 1.3671008503024495e-07
43
pose_error: 0.0
44
pose_error: 0.0
45
pose_error: 1.0542213857434035e-07
46
pose_error: 0.0
47
pose_error: 0.0
48
p

In [13]:
#HTML(html.render(env.sys.replace(dt=env.dt), rollout))

Now testing pose error


In [14]:
state.pipeline_state.x.pos

Array([[1.2526495 , 1.4705337 , 0.814083  ],
       [1.2422284 , 1.4458387 , 1.0487078 ],
       [1.3098793 , 1.4600848 , 1.2616607 ],
       [1.2978325 , 1.2784042 , 1.2982551 ],
       [1.1778704 , 1.147309  , 1.0886564 ],
       [1.2879409 , 1.6438701 , 1.2769486 ],
       [1.1056708 , 1.7849404 , 1.1273315 ],
       [1.2513347 , 1.3861293 , 0.80514073],
       [1.4441509 , 1.3474301 , 0.4322797 ],
       [1.2118335 , 1.2725887 , 0.10300654],
       [1.2539644 , 1.5549382 , 0.8230252 ],
       [1.3402779 , 1.723362  , 0.44634968],
       [1.2488658 , 1.8731027 , 0.07592425]], dtype=float32)

In [15]:
def loss_l2_relpos(pos, ref_pos):
    relpos, ref_relpos = (pos - pos[0])[1:], (ref_pos - ref_pos[0])[1:]
    relpos_loss = (((relpos - ref_relpos) ** 2).sum(-1) ** 0.5).mean()
    return relpos_loss



In [16]:
#using the same since, I am testing with fw kinematics when is the same movement

pose_err = loss_l2_relpos(state.pipeline_state.x.pos,state.pipeline_state.x.pos)
print(pose_err)

0.0


Error in the punch it should be 0 when montion the same. However
I am not getting zero not sure why

In [17]:
# Define the matrices
global_pos = jp.array([
    [0.40274766, 0.36927953, 0.752072],
    [0.4418757, 0.38021225, 0.9847021],
    [0.52255535, 0.4276182, 1.1881032],
    [0.6083375, 0.26487905, 1.2135879],
    [0.44772425, 0.08161816, 1.0865946],
    [0.41400197, 0.57524705, 1.2183353],
    [0.38929832, 0.7546403, 1.0116562],
    [0.45658457, 0.30392218, 0.7460882],
    [0.6134719, 0.4203004, 0.37253296],
    [0.2707621, 0.23714691, 0.24215575],
    [0.34891075, 0.4346369, 0.75805575],
    [0.41676688, 0.5241024, 0.35173994],
    [0.2569707, 0.24560517, 0.09698921]
])

ref_pos = jp.array([
    [0.40274766, 0.36927953, 0.752072],
    [0.44187573, 0.38021225, 0.9847021],
    [0.52255535, 0.4276182, 1.1881032],
    [0.6083376, 0.26487902, 1.2135879],
    [0.4477243, 0.0816181, 1.0865946],
    [0.414002, 0.57524705, 1.2183353],
    [0.38929835, 0.75464034, 1.0116562],
    [0.45658457, 0.30392215, 0.7460882],
    [0.613472, 0.42030036, 0.3725328],
    [0.270762, 0.23714684, 0.24215555],
    [0.34891075, 0.43463692, 0.75805575],
    [0.41676688, 0.52410245, 0.35173988],
    [0.2569707, 0.24560517, 0.09698913]
])

# Check if they are close enough to be considered the same
are_same = jp.allclose(global_pos, ref_pos)

print("Are the two matrices the same?", are_same)

Are the two matrices the same? True


In [18]:
pose_err = loss_l2_relpos(global_pos,ref_pos)
print(pose_err)

7.143896e-08


In [19]:
#idea to deal with this weird errors

tolerance = 1e-6  # This tolerance can be adjusted as needed.
if pose_err < tolerance:
    print("The error is effectively zero.")
else:
    print("The error is significant.")

The error is effectively zero.
